In [1]:
!git clone https://github.com/jjordanoc/CUT3R.git /root/jupyter/CUT3R
%cd /root/jupyter/CUT3R

fatal: destination path '/root/jupyter/CUT3R' already exists and is not an empty directory.
/__modal/volumes/vo-05VlmA5Lclgm1HklUu8tDq/CUT3R


/usr/local/lib/python3.10/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [2]:
# Install PyTorch with CUDA support (Colab already has CUDA 12.x)
!pip install torch torchvision --index-url https://download.pytorch.org/whl/cu121

# Install requirements
!pip install -r requirements.txt

# Install additional dependencies
!pip install git+https://github.com/nerfstudio-project/gsplat.git
!pip install evo open3d


Looking in indexes: https://download.pytorch.org/whl/cu121

[notice] A new release of pip is available: 23.2.1 -> 25.2
[notice] To update, run: pip install --upgrade pip
  Cloning https://github.com/nerfstudio-project/gsplat.git to /tmp/pip-req-build-aw2prgku
  Running command git clone --filter=blob:none --quiet https://github.com/nerfstudio-project/gsplat.git /tmp/pip-req-build-aw2prgku
  Resolved https://github.com/nerfstudio-project/gsplat.git to commit 65042cc501d1cdbefaf1d6f61a9a47575eec8c71
  Running command git submodule update --init --recursive -q
  Installing build dependencies ... done
  Getting requirements to build wheel ... error
  error: subprocess-exited-with-error
  
  × Getting requirements to build wheel did not run successfully.
  │ exit code: 1
  ╰─> [19 lines of output]
      Setting MAX_JOBS to 10
      Traceback (most recent call last):
        File "/usr/local/lib/python3.10/site-packages/pip/_vendor/pyproject_hooks/_in_process/_in_process.py", line 353, in <m

In [3]:
%ls

LICENSE           assets/               demo.py     examples/
README.md         cloud_opt/            demo_ga.py  requirements.txt
__pycache__/      config/               docs/       src/
add_ckpt_path.py  datasets_preprocess/  eval/       viser_utils.py


In [4]:
# Navigate to the CUT3R directory
#%cd /CUT3R
# Apply the fix using sed
!sed -i 's/tokens\.type()/tokens.scalar_type()/g' src/croco/models/curope/kernels.cu

# Rebuild the extension
%cd src/croco/models/curope/
!python setup.py build_ext --inplace


/__modal/volumes/vo-05VlmA5Lclgm1HklUu8tDq/CUT3R/src/croco/models/curope
running build_ext
/usr/local/lib/python3.10/site-packages/torch/utils/cpp_extension.py:495: UserWarning: Attempted to use ninja as the BuildExtension backend but we could not find ninja.. Falling back to using the slow distutils backend.
  warnings.warn(msg.format('we could not find ninja.'))
/usr/local/lib/python3.10/site-packages/torch/utils/cpp_extension.py:424: UserWarning: There are no clang++ version bounds defined for CUDA version 12.1
  warnings.warn(f'There are no {compiler_name} version bounds defined for CUDA version {cuda_str_version}')
building 'curope' extension
clang -pthread -Wno-unused-result -Wsign-compare -Wunreachable-code -DNDEBUG -g -fwrapv -O3 -Wall -fPIC -I/tools/deps/include -I/tools/deps/include/ncursesw -I/tools/deps/libedit/include -g0 -fPIC -I/usr/local/lib/python3.10/site-packages/torch/include -I/usr/local/lib/python3.10/site-packages/torch/include/torch/csrc/api/include -I/usr/local

In [5]:
import os
if not os.path.exists("/root/jupyter/CUT3R/src/cut3r_512_dpt_4_64.pth"):
    %cd /root/jupyter/CUT3R/src
    !pip install gdown
    !gdown --fuzzy https://drive.google.com/file/d/1Asz-ZB3FfpzZYwunhQvNPZEUA8XUNAYD/view?usp=drive_link

In [12]:
import os
from PIL import Image
from pathlib import Path
import shutil

src_dirs = [
    "/root/jupyter/data/iphone13",
    "/root/jupyter/data/iphone16",
    "/root/jupyter/data/samsung-s35-5g",
]
dst_dir = Path("/root/jupyter/data/all")
dst_dir.mkdir(parents=True, exist_ok=True)

def resize_to_max512(img: Image.Image) -> Image.Image:
    w, h = img.size
    if max(w, h) <= 512:
        return img  # no resize needed
    scale = 512 / max(w, h)
    new_size = (int(w * scale), int(h * scale))
    return img.resize(new_size, Image.LANCZOS)

for src in src_dirs:
    for path in Path(src).rglob("*"):
        if path.suffix.lower() in {".jpg", ".jpeg", ".png"}:
            try:
                img = Image.open(path)
                img = resize_to_max512(img)
                # keep filename but flatten structure
                out_path = dst_dir / path.name
                img.save(out_path)
                img.close()
            except Exception as e:
                print(f"❌ Skipped {path}: {e}")

In [ ]:
from datetime import datetime

# Generate human-readable timestamp
timestamp = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
output_dir = f"/root/jupyter/reconstruction/{timestamp}"

seq_path = "/root/jupyter/data/all"



%cd /root/jupyter/CUT3R/
!python demo_ga.py --model_path src/cut3r_512_dpt_4_64.pth --size 512 --seq_path {seq_path} --vis_threshold 1.5 --output_dir {output_dir}

/usr/local/lib/python3.10/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


/__modal/volumes/vo-05VlmA5Lclgm1HklUu8tDq/CUT3R
Found 2737 images in /root/jupyter/data/all.
Preparing input views...
>> Loading a list of 2737 images
 - adding /root/jupyter/data/all/100.jpeg with resolution 512x384 --> 512x384
 - adding /root/jupyter/data/all/1000.jpeg with resolution 512x384 --> 512x384
 - adding /root/jupyter/data/all/1001.jpeg with resolution 512x384 --> 512x384
 - adding /root/jupyter/data/all/1002.jpeg with resolution 512x384 --> 512x384
 - adding /root/jupyter/data/all/1003.jpeg with resolution 512x384 --> 512x384
 - adding /root/jupyter/data/all/1004.jpeg with resolution 512x384 --> 512x384
 - adding /root/jupyter/data/all/1005.jpeg with resolution 512x384 --> 512x384
 - adding /root/jupyter/data/all/1006.jpeg with resolution 512x384 --> 512x384
 - adding /root/jupyter/data/all/1007.jpeg with resolution 512x384 --> 512x384
 - adding /root/jupyter/data/all/1008.jpeg with resolution 512x384 --> 512x384
 - adding /root/jupyter/data/all/1009.jpeg with resolution 

In [ ]:
!python demo.py --model_path src/cut3r_512_dpt_4_64.pth --size 512 --seq_path data/video.mp4 --vis_threshold 1.5 --output_dir reconstruction

2025-10-11 01:54:35.973177: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1760147676.216456   11519 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1760147676.282184   11519 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1760147676.777319   11519 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1760147676.777358   11519 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1760147676.777362   11519 computation_placer.cc:177] computation placer alr

In [ ]:
#!cat demo_ga.py
#!git pull

#!/usr/bin/env python3
"""
3D Point Cloud Inference and Visualization Script

This script performs inference using the ARCroco3DStereo model and visualizes the
resulting 3D point clouds with the PointCloudViewer. Use the command-line arguments
to adjust parameters such as the model checkpoint path, image sequence directory,
image size, device, etc.

Usage:
    python demo_ga.py [--model_path MODEL_PATH] [--seq_path SEQ_PATH] [--size IMG_SIZE]
                            [--device DEVICE] [--vis_threshold VIS_THRESHOLD] [--output_dir OUT_DIR]

Example:
    python demo_ga.py --model_path src/cut3r_512_dpt_4_64.pth \
        --seq_path examples/001 --device cuda --size 512
"""

import os
import numpy as np
import torch
import time
import glob
import random
import cv2
import argparse
import tempfile
import shutil
from copy import deepcopy
from add_ckpt_path import add_path_to_dust3r
import imageio.v2 as iio

# Set random seed for reproducibility.
random.seed(42)

def forward_backward_permu